<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/UserTimePyomoNeos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Accessing computational time in PYOMO with NEOS

## Requirements

In [6]:
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc
import os
import random
import pyomo.environ as pe
os.environ['NEOS_EMAIL'] = 'salvapineda2@gmail.com'

## Create the optimization model in pyomo

In [7]:
# Dimensions
nvar = 10
ncon = 5
# Random vector and matrixes
c = [round(random.uniform(0,1),2) for i in range(nvar)]
A = [[round(random.uniform(-1,1),2) for i in range(nvar)] for j in range(ncon)]
b = [round(random.uniform(-10,10),2) for j in range(ncon)]
# Model
m = pe.ConcreteModel()
# Sets
m.i = pe.Set(initialize=range(nvar),ordered=True)
m.j = pe.Set(initialize=range(ncon),ordered=True)
# Variables
m.x = pe.Var(m.i,within=pe.NonNegativeIntegers)
# Objective function
def obj_rule(m):
  return sum(c[i]*m.x[i] for i in m.i)
m.obj = pe.Objective(rule=obj_rule)
# Constraints
def con_rule(m,j):
  return sum(A[j][i]*m.x[i] for i in m.i) <= b[j]
m.con = pe.Constraint(m.j,rule=con_rule)

## Solve the model using CBC without NEOS

In [8]:
pe.SolverFactory('cbc', executable='/usr/bin/cbc').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 3.49
  Upper bound: 3.49
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 10
  Number of binary variables: 0
  Number of integer variables: 10
  Number of nonzeros: 10
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
  

## Solve the model using CBC and NEOS

In [9]:
pe.SolverManagerFactory('neos').solve(m,opt='cbc').write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 10
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: CBC 2.10.3 optimal, objective 3.49; 0 nodes, 2 iterations, 0.00389 seconds
  Termination condition: optimal
  Id: 0
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
